### 추출한 음식 리스트로 음식 필터링

In [1]:
rainy_day_foods = [
    "파전", "김치찌개", "부침개", "라면", "순대국", "삼계탕", "떡볶이", "우동", "칼국수", "전골",
    "감자전", "김치전", "해물파전", "짜장면", "짬뽕", "감자탕", "순두부찌개", "된장찌개", "고등어조림", "갈비탕",
    "설렁탕", "육개장", "닭볶음탕", "오뎅탕", "매운탕", "아귀찜", "백숙", "수제비", "콩나물국밥", "북엇국",
    "미역국", "도가니탕", "대구탕", "김밥", "비빔밥", "잡채", "갈비찜", "족발", "보쌈", "떡국",
    "만둣국", "어묵", "부대찌개", "곰탕", "청국장찌개", "제육볶음", "닭갈비", "곱창전골", "닭한마리", "오징어볶음",
    "낙지볶음", "양념게장", "간장게장", "감자수제비", "버섯전골", "두부김치", "호박전", "동태찌개", "오삼불고기", "참치김치찌개",
    "비지찌개", "갈치조림", "된장국", "쭈꾸미볶음", "숯불갈비", "꽁치조림", "닭곰탕", "닭죽", "장어구이", "매운갈비찜",
    "홍합탕", "매운오징어볶음", "알탕", "두루치기", "고추장찌개", "오리주물럭", "민어탕", "동태전", "명란젓", "북어찜",
    "보리밥", "콩국수", "황태해장국", "메밀전병", "칼비빔국수", "수육", "닭도리탕", "아구탕", "모둠전", "김치볶음밥",
    "굴국밥", "동치미국수", "고추장불고기", "닭안심찜", "새우튀김", "미나리무침", "어묵볶음", "버섯볶음", "멸치볶음", "가지볶음"
]


In [2]:
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from collections import Counter
import pandas as pd

# 네이버 API 인증 정보
client_id = 'JUIsxNlObCc7b3BCqWe1'
client_secret = 'pqVF4o6iit'

def get_blog_content(query, display):
    url = f"https://openapi.naver.com/v1/search/blog?query={query}&display={display}&sort=sim"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()['items']
    else:
        print("Error Code:", response.status_code)
        return None

def extract_food_names(text_list):
    okt = Okt()
    all_nouns = []

    for text in text_list:
        nouns = okt.nouns(text)
        all_nouns.extend(nouns)
    
    # 음식과 관련된 단어를 필터링합니다.
    foods = [noun for noun in all_nouns if len(noun) > 1]
    # any() -> 정확히 일치할 필요없이 부분 문자열이 포함되면 필터링 하게
    food_names = [name for name in foods if any(food in name for food in rainy_day_foods)]

    return food_names

def main():
    query = '비오는 날 음식'
    blog_posts = get_blog_content(query, 100)
    
    if not blog_posts:
        return None
    
    all_text = []
    
    for post in blog_posts:
        title = post['title']
        description = post['description']
        soup = BeautifulSoup(title + ' ' + description, 'html.parser')
        text = soup.get_text()
        all_text.append(text)
    
    food_names = extract_food_names(all_text)
    
    food_counter = Counter(food_names)
    
    # 데이터 프레임 생성
    food_df = pd.DataFrame(food_counter.items(), columns=['Food', 'Count'])
    
    return food_df

if __name__ == "__main__":
    food_df = main()
    if food_df is not None:
        print(food_df)


     Food  Count
0      어묵      8
1      우동     10
2     된장국      2
3    해물파전     18
4      파전     14
5     감자전     17
6     부침개     38
7     수제비     32
8    두부김치      1
9     칼국수     11
10    짜장면      2
11     짬뽕      3
12     곰탕      2
13    김치전     24
14  김치부침개     11
15     라면      2
16    갈비찜      1
17   김치찌개      1
18    삼계탕      1
19    닭갈비      2
20   만두전골      3


In [3]:
food_df.sort_values('Count', ascending=False)

,Food,Count
6,부침개,38
7,수제비,32
13,김치전,24
3,해물파전,18
5,감자전,17
4,파전,14
14,김치부침개,11
9,칼국수,11
1,우동,10
0,어묵,8


### 12,000가지의 음식이름 데이터와 유사도 검사를 통해 음식이름만 추출하기

In [4]:
import pandas as pd

data = pd.read_csv('data1/식품의약품안전처_통합식품영양성분정보(음식)_20240416.csv', encoding='CP949')
data.head(3)

,식품코드,식품명,데이터구분코드,데이터구분명,식품기원코드,식품기원명,식품대분류코드,식품대분류명,대표식품코드,대표식품명,...,포화지방산(g),트랜스지방산(g),출처코드,출처명,식품중량,업체명,데이터생성방법코드,데이터생성방법명,데이터생성일자,데이터기준일자
0,D219-711000000-0287,아이스크림_프릳츠 커피 아이스크림,D,음식,2,외식(프랜차이즈 등 업체 제공 영양정보),19,유제품류 및 빙과류,19711,아이스크림,...,6.09,0.00,3,식품의약품안전처,115g,배스킨라빈스,2,수집,2023-10-31,2024-04-16
1,D303-148000000-0001,라면_간편조리세트_똠양꿍요괴라면,D,음식,3,외식(분석함량),3,면 및 만두류,3148,라면,...,1.35,0.01,3,식품의약품안전처,1284g,GS 리테일 심플리쿡,1,분석,2022-06-30,2024-04-16
2,D101-004160000-0001,국밥_돼지머리,D,음식,1,가정식(분석 함량),1,밥류,1004,국밥,...,1.47,0.03,3,식품의약품안전처,900g,해당없음,1,분석,2017-12-31,2024-04-16


In [12]:
len(data['식품명'].tolist())

12151

In [5]:
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from concurrent.futures import ThreadPoolExecutor
import time

# 네이버 API 인증 정보
client_id = 'JUIsxNlObCc7b3BCqWe1'
client_secret = 'pqVF4o6iit'

def get_blog_content(query, display):
    url = f"https://openapi.naver.com/v1/search/blog?query={query}&display={display}&sort=sim"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()['items']
    else:
        print("Error Code:", response.status_code)
        return None

def extract_food_names(text_list):
    okt = Okt()
    all_nouns = []

    for text in text_list:
        nouns = okt.nouns(text)
        all_nouns.extend(nouns)
    
    # 음식과 관련된 단어를 필터링합니다.
    foods = [noun for noun in all_nouns if len(noun) > 1]
    # any() -> 정확히 일치할 필요없이 부분 문자열이 포함되면 필터링 하게
    food_names = [name for name in foods if any(food in name for food in rainy_day_foods)]
    
    return food_names

def fetch_and_extract_text(post):
    title = post['title']
    description = post['description']
    soup = BeautifulSoup(title + ' ' + description, 'html.parser')
    return soup.get_text()

def main():
    start_time = time.time()
    query = '비오는 날 음식'
    blog_posts = get_blog_content(query, 100)
    
    if not blog_posts:
        return None
    
    with ThreadPoolExecutor() as executor:
        all_text = list(executor.map(fetch_and_extract_text, blog_posts))
    
    food_names = extract_food_names(all_text)
    
    food_counter = Counter(food_names)
    food_df = pd.DataFrame(food_counter.items(), columns=['Food', 'Count'])
    
    # 음식 메뉴 데이터
    menu_data = data['식품명'].tolist()

    # TF-IDF 벡터라이저를 사용하여 메뉴 데이터를 벡터화
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(menu_data)

    # 블로그 텍스트에서 명사 추출
    blog_text = " ".join(food_names)
    blog_tfidf = vectorizer.transform([blog_text])

    # CPU 코어 수에 맞게 병렬 처리
    with ThreadPoolExecutor() as executor:
        cosine_similarities = list(executor.map(lambda i: cosine_similarity(blog_tfidf, tfidf_matrix[i:i+1]).flatten()[0], range(len(menu_data))))

    # 유사도가 높은 명사 필터링 (임계값 설정, 0.1 ~ 0.3)
    threshold = 0.1
    similar_nouns = [menu_data[i] for i in range(len(cosine_similarities)) if cosine_similarities[i] > threshold]

    # 유사한 명사들만 포함하는 데이터프레임 생성
    similar_food_df = food_df[food_df['Food'].isin(similar_nouns)]

    print("유사한 명사들로 구성된 데이터프레임:")

    print(f"Execution time: {time.time() - start_time} seconds")
    return similar_food_df

if __name__ == "__main__":
    similar_food_df = main()
    if similar_food_df is not None:
        print(similar_food_df)

유사한 명사들로 구성된 데이터프레임:
Execution time: 23.50384211540222 seconds
   Food  Count
1    우동     10
4    파전     14
5   감자전     17
7   수제비     32
9   칼국수     11
13  김치전     24


In [10]:
# 유사도 검사를 통해 추출한 음식들
similar_food_df.sort_values('Count', ascending=False)

,Food,Count
7,수제비,32
13,김치전,24
5,감자전,17
4,파전,14
9,칼국수,11
1,우동,10


In [9]:
# 리스트업한 메뉴들로 필터링한 음식들
food_df.sort_values('Count', ascending=False)

,Food,Count
6,부침개,38
7,수제비,32
13,김치전,24
3,해물파전,18
5,감자전,17
4,파전,14
14,김치부침개,11
9,칼국수,11
1,우동,10
0,어묵,8


=> 유사도 검사를 통해 음식 이름 필터링을 하면 누락되는 음식들이 많다.